In [ ]:
# Import packages
import os
import re  # regular expressions
import warnings
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma
import rasterio as rio
from rasterio.plot import plotting_extent
import geopandas as gpd
import earthpy as et
import earthpy.plot as ep
import earthpy.spatial as es
import earthpy.mask as em

import rioxarray as rxr

from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint

from sklearn.datasets import dump_svmlight_file

from glob import glob

from queue import Queue

warnings.simplefilter('ignore')

# Set working directory
os.chdir(os.path.join(et.io.HOME, 'BD', 'BA_DATA'))

In [ ]:
# Constants
d_limit = 1200
tem_conversion = 0.02 # convert to Kelvin temperature
vege_fillvalue = -3000
vege_lowerest = -2000
NE_fillvalue = 32767
tem_fillvalue = 0
radius = 50

vege_len = 457
tem_len = 914
NE_len = 914
thermal_len = 914

In [ ]:
vege_fs84 = glob('VegeData_16day/*h08v04*')
vege_fs85 = glob('VegeData_16day/*h08v05*')

In [ ]:
tem_fs84 = glob('TemData_8day/*h08v04*')
tem_fs85 = glob('TemData_8day/*h08v05*')

In [ ]:
NE_fs84 = glob('NE_Data_8day/*h08v04*')
NE_fs85 = glob('NE_Data_8day/*h08v05*')

In [ ]:
thermal_fs84 = glob('ThermalData_8day/*h08v04*')
thermal_fs85 = glob('ThermalData_8day/*h08v05*')

In [ ]:
len(NE_fs85)

In [ ]:
len(NE_fs84)

In [ ]:
len(tem_fs85)

### Mask function

Pick any location, divide the neighborhood into 10 regions of interest in various directions, each region is within 50 km, 36 degree and 50 km sector as a region. 

In [ ]:
def sector_mask(shape,centre,radius,angle_range):
    """
    Return a boolean mask for a circular sector. The start/stop angles in  
    `angle_range` should be given in clockwise order.
    """

    x,y = np.ogrid[:shape[0],:shape[1]]
    cx,cy = centre
    tmin,tmax = np.deg2rad(angle_range)

    # ensure stop angle > start angle
    if tmax < tmin:
            tmax += 2*np.pi

    # convert cartesian --> polar coordinates
    r2 = (x-cx)*(x-cx) + (y-cy)*(y-cy)
    theta = np.arctan2(x-cx,y-cy) - tmin

    # wrap angles between 0 and 2*pi
    theta %= (2*np.pi)

    # circular mask
    circmask = r2 <= radius*radius

    # angular mask
    anglemask = theta <= (tmax-tmin)

    return circmask*anglemask

In [ ]:
def compute_feature_set(matrix, fill_value, new_value, row, colmn):
    """
    Return a feature set of a single pixel with row number ROW and column number COLUMN in the 1200 * 1200 matrix MATRIX.
    Feature set is computed based on the average value in 10 regions in one time frame from FILENAMES.
    Fill_value will be replaced by new_value in the feature matrix.
    """
    feature_set = []
    for i in range(10):
        mask = sector_mask(matrix.shape, (row, column), radius, (i, i + 36))
        sector = matrix[mask]
        sector[:][sector[:] == fill_value] = new_value
        mean_value = np.mean(sector)
        feature_set.append(mean_value)
    return feature_set

In [ ]:
def compute_vege_feature_set(matrix, row, colmn):
    """
    Return a vegetation indices feature set of a single pixel with row number ROW and column number COLUMN 
    in the 1200 * 1200 matrix MATRIX.
    Feature set is computed based on the average value in 10 regions in one time frame from FILENAMES.
    """
    feature_set = []
    for i in range(10):
        mask = sector_mask(matrix.shape, (row, column), radius, (i, i + 36))
        sector = matrix[mask]
        sector[:][sector[:] == vege_fillvalue] = vege_lowerest # change fillvalue
        mean_value = np.mean(sector)
        feature_set.append(mean_value)
    return feature_set

In [ ]:
def compute_thermal_feature_set(matrix, row, column):
    """
    Return a thermal anomalies feature set of a single pixel with row number ROW and column number COLUMN 
    in the 1200 * 1200 matrix MATRIX.
    Feature set is computed based on the sum of value in 10 regions in one time frame from FILENAMES.
    """
    feature_set = []
    for i in range(10):
        mask = sector_mask(matrix.shape, (row, column), radius, (i, i + 36))
        sector = matrix[mask]
        sector[:][sector[:] < 7] = 0
        sector[:][sector[:] >= 7] = 1
        sum_value = np.sum(sector)
        feature_set.append(sum_value)
    return feature_set

In [ ]:
def compute_tem_feature_set(matrix, row, column):
    """
    Return a surface temperature feature set of a single pixel with row number ROW and column number COLUMN 
    in the 1200 * 1200 matrix MATRIX.
    Feature set is computed based on the mean of value in 10 regions in one time frame from FILENAMES.
    """
    feature_set = [] # use array instead
    for i in range(10):
        mask = sector_mask(matrix.shape, (row, column), radius, (i, i + 36))
        sector = matrix[mask]
        #sector = sector[sector[:] != 0] # drop fillvalue
        mean_value = np.mean(sector)
        feature_set.append(mean_value)
    return feature_set

In [ ]:
def compute_NE_feature_set(matrix, row, column):
    """
    Return a NE feature set of a single pixel with row number ROW and column number COLUMN 
    in the 1200 * 1200 matrix MATRIX.
    Feature set is computed based on the mean of value in 10 regions in one time frame from FILENAMES.
    """
    feature_set = []
    for i in range(10):
        mask = sector_mask(matrix.shape, (row, column), radius, (i, i + 36))
        sector = matrix[mask]
        mean_value = np.mean(sector)
        feature_set.append(mean_value)
    return feature_set

### Process 16-day hdf file

16-day for vegetation_data

In [ ]:
def vege_features(file, sds, fill_value, new_value, radius):
    """
    Return feature sets of all pixels in the file FILE.
    SDS is NDVI or EVI.
    
    """
    all_bands = []
    with rio.open(file) as dataset:
        for name in dataset.subdatasets:
            if re.search(sds, name):
                with rio.open(name) as subdataset:
                    modis_meta = subdataset.profile
                    all_bands.append(subdataset.read(1))
    vege_modis = np.stack(all_bands)
    vege_matrix = vege_modis[0].reshape(d_limit,d_limit)
    feature_sets = np.zeros((d_limit, d_limit))
    for r in range(d_limit):
        for c in range(d_limit):
            temp = compute_feature_set(vege_martrix, fill_value, new_value, r, c)
            feature_sets[r][c] = temp

In [ ]:
def compute_vege_features(filenames, num_file, sds, fill_value, new_value, radius):
    """
    Return vegetation indice feature sets for all files in FILENAMES
    """
    all_feature_sets = np.zeros((num_file, d_limit, d_limit))
    
    index = 0
    
    for file in filenames:
        temp = vege_features(file, sds, file_value, new_value, radius)
        all_feature_sets[index] = temp
        index += 1
    return all_feature_sets

### Reduce dimension

In [ ]:
def rebin(m, shape):
    """
    Reshape the input matrix A to the shape SHAPE.
    """
    sh = shape[0],m.shape[0]//shape[0],shape[1],m.shape[1]//shape[1]
    return m.reshape(sh).mean(-1).mean(1)

### Process 8-day files

8-day for thermal_data, tem_data and NE_data

In [ ]:
NE_fs85[0:4]

In [ ]:
NE_fs85[4:8]

In [ ]:
def compute_features_8day(filenames, num_file, radius, feature_type):
    """
    Combine every two 8-day files to compute feature sets. There are in total NUM_FILES in filenames. Used for 8-day datasets.
    Use a queue to help iterate through every two files.
    Input filenames is the result from using glob command
    Result is a 3d np array with all the SDS features using 10-region method
    """
    print(1)
    sds1 = None
    sds2 = None
    if feature_type == "thermal":
        sds1 = "FireMask"
    elif feature_type == "tem":
        sds1 = "LST_Day"
    elif feature_type == "NE":
        sds1 = ":ET_500m"
        sds2 = "PET"
    else:
        print("No such feature for 8-day")
        return
    print(2)
    file_count = num_file // 2
    all_feature_sets = np.zeros((file_count, 1080, 1080, 10))
    q = Queue(1000)
    print(3)
    for filename in filenames:
        q.put(filename)
    print(4)
    for i in range (file_count):
        f1 = q.get()
        f2 = q.get()
        f1_bands = []
        f2_bands = []
        f1_bands_se = []
        f2_bands_se = []
        # open two files in a round
        with rio.open(f1) as dataset:
            for name in dataset.subdatasets:
                if re.search(sds1, name):
                    with rio.open(name) as subdataset:
                        modis_meta = subdataset.profile
                        f1_bands.append(subdataset.read(1))
                if feature_type == "NE":
                    if re.search(sds2, name):
                        with rio.open(name) as subdataset:
                            modis_meta = subdataset.profile
                            f1_bands_se.append(subdataset.read(1))
        f1_modis = np.stack(f1_bands)
        f1_matrix = f1_modis[0].reshape(d_limit,d_limit)
        if feature_type == "NE":
            f1_modis_se = np.stack(f1_bands_se)
            f1_matrix_se = f1_modis_se[0].reshape(d_limit,d_limit)
        print(5)
        with rio.open(f2) as dataset:
            for name in dataset.subdatasets:
                if re.search(sds1, name):
                    with rio.open(name) as subdataset:
                        modis_meta = subdataset.profile
                        f2_bands.append(subdataset.read(1))
                if feature_type == "NE":
                    if re.search(sds2, name):
                        with rio.open(name) as subdataset:
                            modis_meta = subdataset.profile
                            f2_bands_se.append(subdataset.read(1))
        f2_modis = np.stack(f2_bands)
        f2_matrix = f2_modis[0].reshape(d_limit,d_limit)
        if feature_type == "NE":
            f2_modis_se = np.stack(f2_bands_se)
            f2_matrix_se = f2_modis_se[0].reshape(d_limit,d_limit)
        # combine two matrices
        if feature_type == "NE":
            f1_matrix = f1_matrix_se - f1_matrix
            f2_matrix = f2_matrix_se - f2_matrix
        combined_matrix = (f1_matrix + f2_matrix) / 2
        if feature_type == "tem":
            combined_matrix = combined_matrix * tem_conversion
        feature_sets = np.zeros((1080, 1080))
        print(6)
        for r in range(60, 1140):
            for c in range(60, 1140):
                if feature_type == "thermal":
                    temp = compute_thermal_feature_set(combined_matrix, r, c)
                elif feature_type == "tem":
                    temp = compute_tem_feature_set(combined_matrix, r, c)
                    print(temp)
                elif feature_type == "vege":
                    temp = compute_vege_feature_set(combined_matrix, r, c)
                else:
                    temp = compute_NE_feature_set(combined_matrix, r, c)
                feature_sets[r][c] = np.copy(temp)
        
        all_feature_sets[i] = np.copy(feature_sets)
        return all_feature_sets

In [ ]:
def compute_TEMfeatures_8day(filenames, num_file, radius):
    """
    Combine every two 8-day files to compute feature sets. There are in total NUM_FILES in filenames. Used for 8-day datasets.
    Use a queue to help iterate through every two files.
    Input filenames is the result from using glob command
    Result is a 3d np array with all the SDS features using 10-region method
    """
    print(1)
    sds1 = "LST_Day"
    print(2)
    file_count = num_file // 2
    all_feature_sets = np.zeros((file_count, 1080, 1080, 10))
    q = Queue(1000)
    print(3)
    for filename in filenames:
        q.put(filename)
    print(4)
    for i in range (file_count):
        f1 = q.get()
        f2 = q.get()
        f1_bands = []
        f2_bands = []
        f1_bands_se = []
        f2_bands_se = []
        # open two files in a round
        with rio.open(f1) as dataset:
            for name in dataset.subdatasets:
                if re.search(sds1, name):
                    with rio.open(name) as subdataset:
                        modis_meta = subdataset.profile
                        f1_bands.append(subdataset.read(1))
        f1_modis = np.stack(f1_bands)
        f1_matrix = f1_modis[0].reshape(d_limit,d_limit)
        print(5)
        with rio.open(f2) as dataset:
            for name in dataset.subdatasets:
                if re.search(sds1, name):
                    with rio.open(name) as subdataset:
                        modis_meta = subdataset.profile
                        f2_bands.append(subdataset.read(1))
        f2_modis = np.stack(f2_bands)
        f2_matrix = f2_modis[0].reshape(d_limit,d_limit)
        # combine two matrices
        combined_matrix = (f1_matrix + f2_matrix) / 2
        combined_matrix = combined_matrix * tem_conversion
        feature_sets = np.zeros((1080, 1080, 10))
        print(6)
        for r in range(60, 1140):
            for c in range(60, 1140):
                print((r-60,c-60))
                temp = compute_tem_feature_set(combined_matrix, r, c)
                feature_sets[r-60][c-60] = np.copy(temp)
        
        all_feature_sets[i] = np.copy(feature_sets)
        return all_feature_sets

In [ ]:
def compute_THERMALfeatures_8day(filenames, num_file, radius):
    """
    Combine every two 8-day files to compute feature sets. There are in total NUM_FILES in filenames. Used for 8-day datasets.
    Use a queue to help iterate through every two files.
    Input filenames is the result from using glob command
    Result is a 3d np array with all the SDS features using 10-region method
    """
    print(1)
    sds1 = "FireMask"
    print(2)
    file_count = num_file // 2
    all_feature_sets = np.zeros((file_count, 1080, 1080, 10))
    q = Queue(file_count)
    print(3)
    for filename in filenames:
        q.put(filename)
    print(4)
    for i in range (file_count):
        f1 = q.get()
        f2 = q.get()
        f1_bands = []
        f2_bands = []
        f1_bands_se = []
        f2_bands_se = []
        # open two files in a round
        with rio.open(f1) as dataset:
            for name in dataset.subdatasets:
                if re.search(sds1, name):
                    with rio.open(name) as subdataset:
                        modis_meta = subdataset.profile
                        f1_bands.append(subdataset.read(1))
        f1_modis = np.stack(f1_bands)
        f1_matrix = f1_modis[0].reshape(d_limit,d_limit)
        print(5)
        with rio.open(f2) as dataset:
            for name in dataset.subdatasets:
                if re.search(sds1, name):
                    with rio.open(name) as subdataset:
                        modis_meta = subdataset.profile
                        f2_bands.append(subdataset.read(1))
        f2_modis = np.stack(f2_bands)
        f2_matrix = f2_modis[0].reshape(d_limit,d_limit)
        # combine two matrices
        combined_matrix = (f1_matrix + f2_matrix) / 2
        feature_sets = np.zeros((1080, 1080))
        print(6)
        for r in range(60, 1140):
            for c in range(60, 1140):
                print((r-60,c-60))
                temp = compute_thermal_feature_set(combined_matrix, r, c)
                feature_sets[r-60][c-60] = np.copy(temp)
        
        all_feature_sets[i] = np.copy(feature_sets)
        return all_feature_sets

In [ ]:
def compute_NEfeatures_8day(filenames, num_file, radius):
    """
    Combine every two 8-day files to compute feature sets. There are in total NUM_FILES in filenames. Used for 8-day datasets.
    Use a queue to help iterate through every two files.
    Input filenames is the result from using glob command
    Result is a 3d np array with all the SDS features using 10-region method
    """
    print(1)
    sds1 = ":ET_500m"
    sds2 = "PET"
    print(2)
    file_count = num_file // 2
    all_feature_sets = np.zeros((file_count, 1080, 1080, 10))
    q = Queue(num_file)
    print(3)
    for filename in filenames:
        q.put(filename)
    print(4)
    for i in range (file_count):
        f1 = q.get()
        f2 = q.get()
        f1_bands = []
        f2_bands = []
        f1_bands_se = []
        f2_bands_se = []
        # open two files in a round
        with rio.open(f1) as dataset:
            for name in dataset.subdatasets:
                if re.search(sds1, name):
                    with rio.open(name) as subdataset:
                        modis_meta = subdataset.profile
                        f1_bands.append(subdataset.read(1))
                if re.search(sds2, name):
                    with rio.open(name) as subdataset:
                        modis_meta = subdataset.profile
                        f1_bands_se.append(subdataset.read(1))
        f1_modis = np.stack(f1_bands)
        f1_matrix = f1_modis[0].reshape(d_limit,d_limit)
        f1_modis_se = np.stack(f1_bands_se)
        f1_matrix_se = f1_modis_se[0].reshape(d_limit,d_limit)
        print(5)
        with rio.open(f2) as dataset:
            for name in dataset.subdatasets:
                if re.search(sds1, name):
                    with rio.open(name) as subdataset:
                        modis_meta = subdataset.profile
                        f2_bands.append(subdataset.read(1))
                if re.search(sds2, name):
                    with rio.open(name) as subdataset:
                        modis_meta = subdataset.profile
                        f2_bands_se.append(subdataset.read(1))
        f2_modis = np.stack(f2_bands)
        f2_matrix = f2_modis[0].reshape(d_limit,d_limit)
        f2_modis_se = np.stack(f2_bands_se)
        f2_matrix_se = f2_modis_se[0].reshape(d_limit,d_limit)
        # combine two matrices
        f1_matrix = f1_matrix_se - f1_matrix
        f2_matrix = f2_matrix_se - f2_matrix
        combined_matrix = (f1_matrix + f2_matrix) / 2
        feature_sets = np.zeros((1080, 1080))
        print(6)
        for r in range(60, 1140):
            for c in range(60, 1140):
                print((r-60,c-60))
                temp = compute_NE_feature_set(combined_matrix, r, c)
                feature_sets[r-60][c-60] = np.copy(temp)
        
        all_feature_sets[i] = np.copy(feature_sets)
        return all_feature_sets

In [ ]:
i = 5
b1 = i >> 0 & 1
b2 = i >> 1 & 1
print(b1)
print(b2)

In [ ]:
def land_mask(file):
    # compute land mask from QA field in thermal data
    all_bands = []
    with rio.open(file) as dataset:
        for name in dataset.subdatasets:
            if re.search("QA", name):
                with rio.open(name) as subdataset:
                    modis_meta = subdataset.profile
                    all_bands.append(subdataset.read(1))
    fire_modis = np.stack(all_bands)
    fire_matrix = fire_modis[0].reshape(d_limit,d_limit)
    for i in range(d_limit):
        for j in range(d_limit):
            k = fire_matrix[i][j]
            b1 = k >> 0 & 1
            b2 = k >> 1 & 1
            if b1 != b2:
                fire_matrix[i][j] = 1
            else:
                fire_matrix[i][j] = 0
    return fire_matrix
lm = land_mask(thermal_fs85[0])

In [ ]:
def random_position():
    random_indexR = np.random.randint(60, 1140)
    random_indexC = np.random.randint(60, 1140)
    return (random_indexR, random_indexC)

In [ ]:
a = random_position()
print(a)

In [ ]:
def compute_features_all(fire_fns, tem_fns, vege_fns, ne_fns, num_file, radius, sample_count):
    """
    X = fire::tem::vege::ne  each row (4 * 10)
    Y = 1/0
    (fire_positions + random 20 samples) rows for X
    """
    print(1)
    fire_sds1 = "FireMask"
    tem_sds1 = "LST_Day"
    vege_sds1 = "EVI"
    ne_sds1 = ":ET_500m"
    ne_sds2 = "PET"
    print(2)
    
    # get fire mask
    print(1)
    sds1 = "FireMask"
    print(2)
    file_count = num_file // 2
    fire_positions = []
    #q = Queue(file_count)
    print(3)
    #for filename in fire_fns:
        #q.put(filename)
    print(4)
    
    # compute fire_combined_matrix
    #f1 = q.get()
    #f2 = q.get()
    f1_bands = []
    f2_bands = []
    # open two files in a round
    with rio.open(fire_fns[0]) as dataset:
        for name in dataset.subdatasets:
            if re.search(fire_sds1, name):
                with rio.open(name) as subdataset:
                    modis_meta = subdataset.profile
                    f1_bands.append(subdataset.read(1))
    f1_modis = np.stack(f1_bands)
    f1_matrix = f1_modis[0].reshape(d_limit,d_limit)
    print(5)
    with rio.open(fire_fns[1]) as dataset:
        for name in dataset.subdatasets:
            if re.search(fire_sds1, name):
                with rio.open(name) as subdataset:
                    modis_meta = subdataset.profile
                    f2_bands.append(subdataset.read(1))
    f2_modis = np.stack(f2_bands)
    f2_matrix = f2_modis[0].reshape(d_limit,d_limit)
    # combine two matrices
    fire_combined_matrix = (f1_matrix + f2_matrix) / 2
    for r in range(60, 1140):
        for c in range(60, 1140):
            if (fire_combined_matrix[r][c] >= 7):
                 fire_positions.append([r,c])
    print("fire_mask")
    #feature_count = len(fire_positions) * 4 * 10
    #all_feature_sets = np.zeros((1080, 1080, 40))
    print(len(fire_positions))
    
    # compute tem_combined_matrix
    f1_bands = []
    f2_bands = []
    with rio.open(tem_fns[0]) as dataset:
        for name in dataset.subdatasets:
            if re.search(tem_sds1, name):
                with rio.open(name) as subdataset:
                    modis_meta = subdataset.profile
                    f1_bands.append(subdataset.read(1))
    f1_modis = np.stack(f1_bands)
    f1_matrix = f1_modis[0].reshape(d_limit,d_limit)
    print(5)
    with rio.open(tem_fns[1]) as dataset:
        for name in dataset.subdatasets:
            if re.search(tem_sds1, name):
                with rio.open(name) as subdataset:
                    modis_meta = subdataset.profile
                    f2_bands.append(subdataset.read(1))
    f2_modis = np.stack(f2_bands)
    f2_matrix = f2_modis[0].reshape(d_limit,d_limit)
    # combine two matrices
    tem_combined_matrix = (f1_matrix + f2_matrix) / 2
    
    
    # compute vege_combined_matrix
    all_bands = []
    with rio.open(vege_fns) as dataset:
        for name in dataset.subdatasets:
            if re.search(vege_sds1, name):
                with rio.open(name) as subdataset:
                    modis_meta = subdataset.profile
                    all_bands.append(subdataset.read(1))
    vege_modis = np.stack(all_bands)
    vege_matrix = vege_modis[0].reshape(d_limit,d_limit)
        
        
    # compute ne_combined_matrix    
    f1_bands = []
    f2_bands = []
    f1_bands_se = []
    f2_bands_se = []
    # open two files in a round
    with rio.open(ne_fns[0]) as dataset:
        for name in dataset.subdatasets:
            if re.search(ne_sds1, name):
                with rio.open(name) as subdataset:
                    modis_meta = subdataset.profile
                    f1_bands.append(subdataset.read(1))
            if re.search(ne_sds2, name):
                with rio.open(name) as subdataset:
                    modis_meta = subdataset.profile
                    f1_bands_se.append(subdataset.read(1))
    f1_modis = np.stack(f1_bands)
    print(f1_modis.shape)
    f1_matrix = rebin(f1_modis[0], (d_limit,d_limit))
    f1_modis_se = np.stack(f1_bands_se)
    f1_matrix_se = rebin(f1_modis_se[0], (d_limit,d_limit))
    print(5)
    with rio.open(ne_fns[1]) as dataset:
        for name in dataset.subdatasets:
            if re.search(ne_sds1, name):
                with rio.open(name) as subdataset:
                    modis_meta = subdataset.profile
                    f2_bands.append(subdataset.read(1))
            if re.search(ne_sds2, name):
                with rio.open(name) as subdataset:
                    modis_meta = subdataset.profile
                    f2_bands_se.append(subdataset.read(1))
    f2_modis = np.stack(f2_bands)
    f2_matrix = rebin(f2_modis[0], (d_limit,d_limit))
    f2_modis_se = np.stack(f2_bands_se)
    f2_matrix_se = rebin(f2_modis_se[0], (d_limit,d_limit))
    # combine two matrices
    f1_matrix = f1_matrix_se - f1_matrix
    f2_matrix = f2_matrix_se - f2_matrix
    ne_combined_matrix = (f1_matrix + f2_matrix) / 2
        
    all_feature_sets = []
    
    # random samples   
    for i in range(sample_count):
        print(i)
        p = random_position()
        while True:
            if (p not in fire_positions) and lm[p[0]][p[1]] == 1:
                break;
            else:
                p = random_position()
                
        # now have a good random position
        r = p[0]
        c = p[1]
        features = []
        thermal_feature_set = compute_thermal_feature_set(fire_combined_matrix, r, c)
        features.append(thermal_feature_set)
        tem_feature_set = compute_thermal_feature_set(tem_combined_matrix, r, c)
        features.append(tem_feature_set)
        vege_feature_set = compute_thermal_feature_set(vege_matrix, r, c)
        features.append(vege_feature_set)
        ne_feature_set = compute_thermal_feature_set(ne_combined_matrix, r, c)
        features.append(ne_feature_set)
        all_feature_sets.append(features)
    
    # fire positions
    final_fire_positions = []
    count = 0
    if sample_count > len(fire_positions):
        count = len(fire_positions)
    else:
        count = sample_count
    print(count)
    for i in range(count):
        print(i)
        p = np.random.randint(0, len(fire_positions))
        while True:
            if fire_positions[p] not in final_fire_positions:
                final_fire_positions.append(fire_positions[p])
                break;
            else:
                p = np.random.randint(0, len(fire_positions))
    for i in final_fire_positions:
        print(i)
        r = i[0]
        c = i[1]
        features = []
        thermal_feature_set = compute_thermal_feature_set(fire_combined_matrix, r, c)
        features.append(thermal_feature_set)
        tem_feature_set = compute_thermal_feature_set(tem_combined_matrix, r, c)
        features.append(tem_feature_set)
        vege_feature_set = compute_thermal_feature_set(vege_matrix, r, c)
        features.append(vege_feature_set)
        ne_feature_set = compute_thermal_feature_set(ne_combined_matrix, r, c)
        features.append(ne_feature_set)
        all_feature_sets.append(features)

        
    return all_feature_sets

In [ ]:
result1 = compute_features_all(thermal_fs85[0:2], tem_fs85[0:2], vege_fs85[0], NE_fs85[0:2], 2, 50, 10)

In [ ]:
print(result1)

Within a region, exact a small set of features, and ask is there any fire observed in the region in the past 16 days. Consider the intensity of fire, number of pixels with fire (a pixel with fire for 3 days are considered as 3 fire), for a region in the past 16 days.

In [ ]:
len(tem_fs85)

In [ ]:
len(vege_fs85)

In [ ]:
Tem = compute_TEMfeatures_8day(tem_fs85[0:4], 4, 50)

### Time Series Features

In [ ]:
X = np.ones((3,4))
Y = np.zeros(3)

In [ ]:
X

In [ ]:
Y.shape

In [ ]:
dump_svmlight_file(X,Y,os.path.join("test_sample1"),zero_based=False)

In [ ]:
XX = np.ones((5,4))
YY = np.ones(5)
dump_svmlight_file(XX,YY,os.path.join("test_sample2"),zero_based=False)

In [ ]:
a = np.append(Y, [4, 5, 6, 7, 8, 9])

In [ ]:
b = np.zeros((3,3,2))

In [ ]:
c = [1.0, 2.1]

In [ ]:
b[0][0]

In [ ]:
b[0][0] = np.copy(c)

In [ ]:
b